Final version of code submission for Vesuvius Challenge - Surface Detection [Link](https://www.kaggle.com/competitions/vesuvius-challenge-surface-detection/)

In [ ]:
#local installation of the dependencies due to competition rules
from IPython.display import clear_output

var="/kaggle/input/vsdetection-packages-offline-installer-only/whls"
!pip install \
  "$var"/keras_nightly-*.whl \
  "$var"/tifffile-*.whl \
  "$var"/imagecodecs-*.whl \
  "$var"/medicai-*.whl \
  --no-index \
  --find-links "$var"

clear_output()

Version computed with P100 over the course of 6 hours

In [2]:
import os
os.environ["KERAS_BACKEND"] = "jax"

import keras
from medicai.transforms import (
    Compose,
    ScaleIntensityRange,
    NormalizeIntensity
)
from medicai.models import SegFormer, TransUNet
from medicai.utils.inference import SlidingWindowInference

import numpy as np
import pandas as pd
import zipfile
import tifffile
import scipy.ndimage as ndi
from skimage.morphology import remove_small_objects
from matplotlib import pyplot as plt

keras.config.backend(), keras.version()

2026-02-04 12:31:53.735197: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770208314.221857      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770208314.367539      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770208315.490131      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770208315.490186      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770208315.490189      24 computation_placer.cc:177] computation placer alr

('jax', '3.12.0.dev2025100703')

In [3]:
root_dir = "/kaggle/input/vesuvius-challenge-surface-detection"
test_dir = f"{root_dir}/test_images"
output_dir = "/kaggle/working/submission_masks"
zip_path = "/kaggle/working/submission.zip"
os.makedirs(output_dir, exist_ok=True)

Each scroll id contains the reference to 3D volume of CT scan represented in a tiff file. Those scans represent the carbonized, tightly placed ancient scrolls. 

In [4]:
test_df = pd.read_csv(f"{root_dir}/test.csv")
test_df.head()

,id,scroll_id
0,1407735,26002


In [5]:
def val_transformation(image):
    data = {"image": image}
    pipeline = Compose([
        NormalizeIntensity(
            keys=["image"], 
            nonzero=True,
            channel_wise=False
        ),
    ])
    result = pipeline(data)
    return result["image"]

The parameters represent num of dimensions as num_classes, input shape of volume compressed to 50% and number of time transformations

In [6]:
tta=1
num_classes=3
input_shape=(160, 160, 160)
kaggle_model_path = "/kaggle/input/vsd-model/keras/"

In [ ]:
def get_model():
    # 0.545 (tta+pp)
    model = TransUNet(
        input_shape=(160, 160, 160, 1),
        encoder_name='seresnext50',
        classifier_activation=None,
        num_classes=3,
    )
    model.load_weights(
        f"{kaggle_model_path}/transunet/3/transunet.seresnext50.160px.comboloss.weights.h5"
    )
    
    return model

In [8]:
model = get_model()
model.count_params() / 1e6

70.056598

In [9]:
model.instance_describe()

Instance of TransUNet
  • input_shape: (160, 160, 160, 1)
  • num_classes: 3
  • num_queries: 100
  • encoder: SEResNeXt50(
    • name: 'SEResNeXt503D'
    • trainable: True
    • input_shape: (160, 160, 160, 1)
    • include_rescaling: False
    )
  • encoder_name: 'seresnext50'
  • encoder_depth: 5
  • classifier_activation: linear
  • num_vit_layers: 12
  • num_heads: 8
  • embed_dim: 512
  • mlp_dim: 1024
  • dropout_rate: 0.1
  • decoder_activation: leaky_relu
  • decoder_filters: (256, 128, 64, 32, 16)
  • encoder: SEResNeXt50(
    • name: 'SEResNeXt503D'
    • trainable: True
    • input_shape: (160, 160, 160, 1)
    • include_rescaling: False
    )

In [3]:
!pip install imagecodecs

   ---------------------------------------- 0.0/21.5 MB ? eta -:--:--
   - -------------------------------------- 0.8/21.5 MB 7.0 MB/s eta 0:00:03
   ---- ----------------------------------- 2.6/21.5 MB 7.3 MB/s eta 0:00:03
   ------- -------------------------------- 4.2/21.5 MB 7.4 MB/s eta 0:00:03
   ---------- ----------------------------- 5.8/21.5 MB 7.4 MB/s eta 0:00:03
   -------------- ------------------------- 7.6/21.5 MB 7.5 MB/s eta 0:00:02
   ---------------- ----------------------- 8.9/21.5 MB 7.5 MB/s eta 0:00:02
   ------------------- -------------------- 10.7/21.5 MB 7.5 MB/s eta 0:00:02
   ---------------------- ----------------- 12.1/21.5 MB 7.5 MB/s eta 0:00:02
   ------------------------ --------------- 13.4/21.5 MB 7.3 MB/s eta 0:00:02
   --------------------------- ------------ 14.9/21.5 MB 7.3 MB/s eta 0:00:01
   ------------------------------ --------- 16.5/21.5 MB 7.3 MB/s eta 0:00:01
   --------------------------------- ------ 17.8/21.5 MB 7.2 MB/s eta 0:00:01


In [1]:
!pip install napari

  Using cached napari-0.6.6-py3-none-any.whl.metadata (17 kB)
  Using cached app_model-0.4.0-py3-none-any.whl.metadata (3.2 kB)
  Using cached cachey-0.2.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached certifi-2026.1.4-py3-none-any.whl.metadata (2.5 kB)
  Using cached dask-2026.1.2-py3-none-any.whl.metadata (3.8 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
  Using cached magicgui-0.10.1-py3-none-any.whl.metadata (5.5 kB)
  Using cached napari_console-0.1.4-py3-none-any.whl.metadata (5.7 kB)
  Using cached napari_plugin_engine-0.2.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached napari_svg-0.2.1-py3-none-any.whl.metadata (3.6 kB)
  Using cached npe2-0.7.9-py3-none-any.whl.metadata (6.0 kB)
  Using cached numpydoc-1.10.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached pint-0.25.2-py3-none-any.whl.metadata (10 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached pyopengl-3.1.10-py3-none-any.whl.metadata (3.3 kB)
  Using cach

Visualizing example volume with Napari (interactive window)

In [ ]:
import napari
from tifffile import imread

data = imread('./data/1407735.tif')
viewer = napari.Viewer()
viewer.add_image(data, name='3D Volume')
viewer.dims.ndisplay = 3
napari.run()

For inferencing final solution of the model we utilized the sliding window over three-dimensional volume with partial overlap

In [10]:
swi = SlidingWindowInference(
    model,
    num_classes=3,
    roi_size=input_shape,
    sw_batch_size=1,
    mode='gaussian',
    overlap=0.40,
)

In [11]:
def load_volume(path):
    vol = tifffile.imread(path)
    vol = vol.astype(np.float32)
    vol = vol[None, ..., None]
    return vol

Building predictions include transforming the data in multiple ways (flipping, rotating in 2 axis)

In [12]:
def predict_with_tta(inputs, swi):
    logits = []

    # Original
    logits.append(swi(inputs))

    # Flips (spatial only)
    for axis in [1, 2, 3]:
        img_f = np.flip(inputs, axis=axis)
        p = swi(img_f)
        p = np.flip(p, axis=axis)
        logits.append(p)

    # Axial rotations (H, W)
    for k in [1, 2, 3]:
        img_r = np.rot90(inputs, k=k, axes=(2, 3))
        p = swi(img_r)
        p = np.rot90(p, k=-k, axes=(2, 3))
        logits.append(p)

    mean_logits = np.mean(logits, axis=0)
    return mean_logits.argmax(-1).astype(np.uint8).squeeze()

Building struct and post processing for turning a probability volume into binary representation

In [13]:
def build_anisotropic_struct(z_radius: int, xy_radius: int):
    z, r = z_radius, xy_radius
    if z == 0 and r == 0:
        return None
    if z == 0 and r > 0:
        size = 2 * r + 1
        struct = np.zeros((1, size, size), dtype=bool)
        cy, cx = r, r
        for dy in range(-r, r + 1):
            for dx in range(-r, r + 1):
                if dy * dy + dx * dx <= r * r:
                    struct[0, cy + dy, cx + dx] = True
        return struct
    if z > 0 and r == 0:
        struct = np.zeros((2 * z + 1, 1, 1), dtype=bool)
        struct[:, 0, 0] = True
        return struct
    depth = 2 * z + 1
    size = 2 * r + 1
    struct = np.zeros((depth, size, size), dtype=bool)
    cz, cy, cx = z, r, r
    for dz in range(-z, z + 1):
        for dy in range(-r, r + 1):
            for dx in range(-r, r + 1):
                if dy * dy + dx * dx <= r * r:
                    struct[cz + dz, cy + dy, cx + dx] = True
    return struct

def topo_postprocess(
    probs,
    T_low=0.90,
    T_high=0.90,
    z_radius=1,
    xy_radius=0,
    dust_min_size=100,
):
    # Step 1: 3D Hysteresis
    strong = probs >= T_high
    weak   = probs >= T_low

    if not strong.any():
        return np.zeros_like(probs, dtype=np.uint8)

    struct_hyst = ndi.generate_binary_structure(3, 3)
    mask = ndi.binary_propagation(
        strong, mask=weak, structure=struct_hyst
    )

    if not mask.any():
        return np.zeros_like(probs, dtype=np.uint8)

    # Step 2: 3D Anisotropic Closing
    if z_radius > 0 or xy_radius > 0:
        struct_close = build_anisotropic_struct(z_radius, xy_radius)
        if struct_close is not None:
            mask = ndi.binary_closing(mask, structure=struct_close)

    # Step 3: Dust Removal
    if dust_min_size > 0:
        mask = remove_small_objects(
            mask.astype(bool), min_size=dust_min_size
        )

    return mask.astype(np.uint8)


Final pipeline and submission

In [14]:
def inference_pipelines(
    volume,
    T_low=0.30,
    T_high=0.80,
    z_radius=3,
    xy_radius=2,
    dust_min_size=223,
):
    probs = predict_with_tta(volume, swi)
    final = topo_postprocess(
        probs,
        T_low=T_low,
        T_high=T_high,
        z_radius=z_radius,
        xy_radius=xy_radius,
        dust_min_size=dust_min_size,
    )
    return final


In [15]:
with zipfile.ZipFile(
    zip_path, "w", compression=zipfile.ZIP_DEFLATED
) as z:
    for image_id in test_df["id"]:
        tif_path = f"{test_dir}/{image_id}.tif"
        
        volume = load_volume(tif_path)
        volume = val_transformation(volume)
        output = inference_pipelines(volume) 
        
        out_path = f"{output_dir}/{image_id}.tif"
        tifffile.imwrite(out_path, output.astype(np.uint8))

        z.write(out_path, arcname=f"{image_id}.tif")
        os.remove(out_path)

print("Submission ZIP:", zip_path)

I0000 00:00:1770208350.008255      24 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2559 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1770208350.013399      24 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13755 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
Total patch 27: 100%|██████████| 27/27 [00:16<00:00,  1.63it/s]


Submission ZIP: /kaggle/working/submission.zip
